# Heat Exchanger Lumped-Capacitance Unit Model with Two Property Packages

![](heat_exchanger_4.svg)

**Problem Statement**: In this example, we will be heating a benzene-toluene mixture using steam. 

**Tube Side Inlet**

Flow Rate = 250 mol/s

Mole fraction (Benzene) = 0.4

Mole fraction (Toluene) = 0.6

Pressure = 101325 Pa

Temperature = 380 K

**Shell Side Inlet**

Flow Rate = 100 mol/s

Mole fraction (Steam) = 1

Pressure = 101325 Pa

Temperature = 400 K

The 0D lumped-capacitance heat exchanger unit model makes the assumption that the temperature difference bewtween discrete "lumps" in the heat exchanger is negligible. Likewise, this model can be useful for transient simulations in which temperature gradients within the heat exchanger can be ignored. This example will demonstrate the simulation of the lumped-capacitance heat exchanger by fixing the following degrees of freedom:
- heat transfer area
- overall heat transfer coefficients (one for cold-side and one for hot-side)

The IDAES library's `HeatExchangerLumpedCapacitance` is modelled after the heat exchanger 0D model and has added the capabilities of supporting a wall temperature as well as heat holdup for transient simulations; more details on the lumped capacitance heat exchanger may be found here: https://idaes-pse.readthedocs.io/en/latest/reference_guides/model_libraries/generic/unit_models/heat_exchanger_lc.html.

IDAES documentation reference for the simpler heat exchanger 0D model: https://idaes-pse.readthedocs.io/en/latest/reference_guides/model_libraries/generic/unit_models/heat_exchanger.html

## Setting up the problem in IDAES

First, import the required IDAES and Pyomo modules. Note that the hot-side (shell) and cold-side (tube) properties leverage separate property packages:

In [ ]:
# Import pyomo package 
from pyomo.environ import ConcreteModel, Constraint, value, units

# Import idaes logger to set output levels
import idaes.logger as idaeslog

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

# Import the IAPWS property package to create a properties block for the flowsheet
from idaes.models.properties import iapws95

from idaes.models.properties.iapws95 import htpx

from idaes.models.properties.modular_properties import GenericParameterBlock

from idaes.models.properties.modular_properties.examples.BT_ideal \
    import configuration

#Import the degrees_of_freedom function from the idaes.core.util.model_statistics package
from idaes.core.util.model_statistics import degrees_of_freedom

#Import the default IPOPT solver
from idaes.core.solvers import get_solver

#Import a heat exchanger unit
from idaes.models.unit_models import HeatExchangerLumpedCapacitance, HeatExchangerFlowPattern

Then, build the model and flowsheet components using the imported property packages:

In [ ]:
# Create the ConcreteModel and the FlowsheetBlock, and attach the flowsheet block to it.
m = ConcreteModel()

# Steady State Model
m.fs = FlowsheetBlock(dynamic=False)

# Setup property packages for shell and tube side
# Steam property package
m.fs.properties_shell = iapws95.Iapws95ParameterBlock()

# BT ideal property package
m.fs.properties_tube = GenericParameterBlock(**configuration)

Then, import and define the `HeatExchanger` unit, and add it to the flowsheet. As mentioned above, we will designate the shell as the hot side and the tube as the cold side. The `dynamic_heat_balance` is set to `False` in this case, meaning heat holdup in the wall material is not considered.

The unit is created below:

In [ ]:
# Create an instance of the heat exchanger unit, attaching it to the flowsheet
# Specify that the property packages to be used with the heat exchanger are the ones we created earlier.
m.fs.heat_exchanger = HeatExchangerLumpedCapacitance(
        hot_side_name="shell",
        cold_side_name="tube",
        shell={"property_package": m.fs.properties_shell},
        tube={"property_package": m.fs.properties_tube},
        flow_pattern=HeatExchangerFlowPattern.crossflow,
        dynamic_heat_balance=False,)

# Call the degrees_of_freedom function, get initial DOF
DOF_initial = degrees_of_freedom(m)
print("The initial DOF is {0}".format(DOF_initial))

In [ ]:
assert DOF_initial == 12

## Fixing input specifications
In the following cell, we will use a function to calculate enthalpy and then specify all of the necessary inputs for the heat exchanger.

In [ ]:
h = htpx(400*units.K, P = 101325*units.Pa)  # calculate IAPWS enthalpy

# Fix the inlet conditions
m.fs.heat_exchanger.shell_inlet.flow_mol.fix(100) # mol/s
m.fs.heat_exchanger.shell_inlet.pressure.fix(101325) # Pa
m.fs.heat_exchanger.shell_inlet.enth_mol.fix(h) # J/mol

DOF_initial = degrees_of_freedom(m)
print("The DOF is {0}".format(DOF_initial))

In [ ]:
m.fs.heat_exchanger.tube_inlet.flow_mol.fix(250) # mol/s
m.fs.heat_exchanger.tube_inlet.mole_frac_comp[0, "benzene"].fix(0.4)
m.fs.heat_exchanger.tube_inlet.mole_frac_comp[0, "toluene"].fix(0.6)
m.fs.heat_exchanger.tube_inlet.pressure.fix(101325) # Pa
m.fs.heat_exchanger.tube_inlet.temperature[0].fix(380) # K

# Call the degrees_of_freedom function, get final DOF
DOF_final = degrees_of_freedom(m)
print("The DOF is {0}".format(DOF_final))

### Fix overall heat transfer coefficient (HTC) and the heat transfer area
Unlike the other heat exchanger unit models which only require one heat transfer coeffcient, the lump-capacitance model requires an overall heat transfer coeffcient to be specified for the hot and cold sides. Below, we fix the heat exchanger area and heat transfer coefficients, which yields a fully defined problem with zero degrees of freedom that may be initialized and solved:

In [ ]:
m.fs.heat_exchanger.area.fix(50) # m2
m.fs.heat_exchanger.ua_hot_side.fix(200 * 1000) # W/m2/K
m.fs.heat_exchanger.ua_cold_side.fix(200 * 1000) # W/m2/K
m.fs.heat_exchanger.crossflow_factor.fix(0.6)

# Call the degrees_of_freedom function, get final DOF
DOF_final = degrees_of_freedom(m)
print("The DOF is {0}".format(DOF_final))

In [ ]:
assert DOF_final == 0

Now that the problem is square (zero degrees of freedom), we can initialize and solve the full model:

In [ ]:
# Initialize the flowsheet, and set the output at INFO
m.fs.heat_exchanger.initialize(outlvl=idaeslog.INFO)

# Solve the simulation using ipopt
# Note: If the degrees of freedom = 0, we have a square problem
opt = get_solver()
solve_status = opt.solve(m)

# Display a readable report
m.fs.heat_exchanger.report()

In [ ]:
from pyomo.environ import assert_optimal_termination
import pytest

# Check if termination condition is optimal
assert_optimal_termination(solve_status)

assert value(m.fs.heat_exchanger.shell.properties_out[0].temperature) == pytest.approx(380.0, abs=1e-2)
assert value(m.fs.heat_exchanger.tube.properties_out[0].temperature) == pytest.approx(382.38, abs=1e-2)